In [ ]:
from dotenv import load_dotenv
from pymatgen.core import Structure
from monty.serialization import dumpfn
import numpy as np

from utils import _wf_dev, _mpdb, get_e_form, comp_from_struct_dict

load_dotenv()

wf_dev_store = _wf_dev()
prod_store = _mpdb()

We first obtains the energies of unary, on the hull structures after relaxation with DFT

In [ ]:
elemental_energies = {}
for doc in wf_dev_store.query(
    {
        "metadata.job_info": "mof discovery dft elemental",
        "name": {"$regex": "r2SCAN\-D4 static"},
    }
):
    structure = Structure.from_dict(doc["output"]["structure"])
    ele = list(structure.composition.as_dict())[0]
    elemental_energies[ele] = doc["output"]["output"]["energy"] / len(structure)
dumpfn(elemental_energies, "unary_energy_per_atom.json.gz")

Similarly, we obtain the final structures and energies of the MOFs after relaxation with DFT

In [ ]:
final_structures = {}

cursors = [
    prod_store.query(
        {
            "metadata.job_info": "mof_discovery_filtered",
            "name": {"$regex": "r2SCAN\-D4 static"},
        }
    ),
    wf_dev_store.query(
        {
            "metadata.job_info": "mof discovery dft",
            "name": {"$regex": "r2SCAN\-D4 static"},
        }
    ),
]

for cursor in cursors:
    for doc in cursor:
        if (mof_id := doc["metadata"]["mof_id"]) in final_structures:
            for i in range(len(final_structures)):
                if (new_id := f"{mof_id}.{i}") not in final_structures:
                    mof_id = new_id
                    break
        final_structures[mof_id] = {
            "structure": doc["output"]["output"]["structure"],
            "energy": doc["output"]["output"]["energy"],
            "bandgap": doc["output"]["output"]["bandgap"],
            "e_form_per_atom": get_e_form(
                doc["output"]["output"]["energy"],
                doc["output"]["output"]["structure"],
                elemental_energies,
            ),
        }
dumpfn(final_structures, "mof_final_structures.json.gz")

In [ ]:
by_comp = {}
mets = set()
for entry in final_structures.values():
    comp = comp_from_struct_dict(entry["structure"])
    metals = set(
        [ele for ele in comp if ele not in ("C", "H", "N", "O", "S", "F", "Cl")]
    )
    mets.update(metals)
    nodes = "-".join(metals)
    if nodes not in by_comp:
        by_comp[nodes] = []
    by_comp[nodes].append(entry["e_form_per_atom"])
print(mets)
for node, e_forms in by_comp.items():
    print(node, len(e_forms))

## Fig. S3(d)

In [ ]:
from mp_api.client import MPRester

with MPRester() as mpr:
    thermo_docs = mpr.materials.thermo.search(
        thermo_types=["R2SCAN"],
        fields=["formation_energy_per_atom", "material_id", "thermo_type"],
    )

In [ ]:
import plotly.graph_objects as pgo
from scipy.stats import gaussian_kde

from seaborn import color_palette

alpha = [
    *[1 for _ in range(3)],
    *[0.8 for _ in range(len(color_palette("colorblind")) - 3)],
]
_non_alpha_colors = [
    f"rgb({','.join(str(255 * f) for f in v)})" for v in color_palette("bright")
]
_colors = [
    f"rgba({cstr},{alpha[i]})"
    for i, cstr in enumerate(
        [",".join(str(f) for f in v) for v in color_palette("colorblind")]
    )
]

colors = {k: _colors[i] for i, k in enumerate(["Zn", "Mg", "Al", "Materials Project"])}

non_alpha_colors = {
    k: _non_alpha_colors[i]
    for i, k in enumerate(["Zn", "Mg", "Al", "Materials Project"])
}

mp_bds = (-2.0, 0.5)

for ifig in range(2):
    for show_bins in (False, True):
        aux_str = "_w_bins" if show_bins else ""

        if ifig == 0:
            sorted_nodes = sorted(
                ["Zn", "Mg", "Al"], key=lambda k: len(by_comp[k]), reverse=True
            )
            # colors = alpha_colors
            fig_name = f"mof_formation_energies{aux_str}.pdf"
        else:
            sorted_nodes = ["Zn", "Materials Project"]
            # colors = non_alpha_colors
            fig_name = f"zn_mof_formation_energies{aux_str}.pdf"

        fig = pgo.Figure()

        bds = (
            min(
                min(e_forms)
                for k, e_forms in by_comp.items()
                if k != "Materials Project"
            ),
            max(
                max(e_forms)
                for k, e_forms in by_comp.items()
                if k != "Materials Project"
            ),
        )

        base_axis_opts = {
            "title_font_size": 32,
            "title_font_color": "black",
            "title_font_family": "Arial",
            "tickfont_color": "black",
            "tickfont_size": 28,
            "showline": True,
            "linewidth": 2,
            "linecolor": "black",
        }

        counts = {}
        bins = {}
        nbin = 25
        for node in sorted_nodes:
            counts[node], bins[node] = np.histogram(
                by_comp[node],
                bins=200
                if node == "Materials Project"
                else nbin,  # int(np.ceil(0.5*len(e_forms))),
                range=mp_bds if node == "Materials Project" else bds,
                density=True,
                weights=None,
            )

        zorders = {node: np.zeros(nbin, dtype=int) for node in sorted_nodes}
        for i in range(nbin):
            ordered = sorted(sorted_nodes, key=lambda k: counts[k][i], reverse=True)
            for iz, node in enumerate(ordered):
                zorders[node][i] = iz + 1

        for inode, node in enumerate(sorted_nodes):
            e_forms = by_comp[node]
            gauss_kde = gaussian_kde(e_forms)

            if node != "Materials Project":
                for z in range(len(sorted_nodes)):
                    count_copy = counts[node].copy()
                    count_copy[zorders[node] != 1 + z] = 0.0

                    if show_bins:
                        fig.add_trace(
                            pgo.Bar(
                                x=bins[node],
                                y=count_copy,
                                name=node,
                                marker={
                                    "color": colors[node],
                                    # "line_color": "white",
                                },
                                showlegend=False,
                                zorder=z,
                            ),
                        )
            else:
                if show_bins:
                    fig.add_trace(
                        pgo.Bar(
                            x=bins[node],
                            y=counts[node],
                            name=node,
                            marker={
                                "color": colors[node],
                                # "line_color": "white",
                            },
                            showlegend=False,
                            zorder=4,
                        ),
                    )

            if ifig == 0:
                itp_bds = (bins[node].min(), bins[node].max())
            else:
                itp_bds = mp_bds
            interp = np.linspace(
                (1.0 - 0.1 * np.sign(itp_bds[0])) * itp_bds[0],
                (1.0 + 0.1 * np.sign(itp_bds[1])) * itp_bds[1],
                2000,
            )
            fig.add_trace(
                pgo.Scatter(
                    x=interp,
                    y=gauss_kde(interp),
                    marker={
                        "color": non_alpha_colors[node],
                    },
                    name=node,
                    mode="lines",
                    showlegend=True,
                    zorder=5,
                    fill=None if show_bins else "tozeroy",
                )
            )

        h = 800
        aspect_ratio = 7 / 5
        fig.update_layout(
            barmode="overlay",
            height=h,
            width=h * aspect_ratio,
            yaxis={
                "title_text": "Density",
                "range": (0, max(max(count) for count in counts.values()))
                if show_bins
                else (0.0, 2.5),
                # "type": "log",
                **base_axis_opts,
            },
            xaxis={
                "title_text": "Formation energy (eV/atom)",
                "linecolor": "black",
                "range": [1.2 * bd for bd in bds] if ifig == 0 else mp_bds,
                **base_axis_opts,
            },
            legend={
                "x": 0.0 if ifig == 0 else 1.0,
                "y": 1.0,
                "yanchor": "top",
                "xanchor": "left" if ifig == 0 else "right",
                "bgcolor": "rgba(0,0,0,0)",
                "font_size": 28,
                "font_color": "black",
                "orientation": "v",
                "title_font_family": "Arial",
            },
            margin={"t": 0, "r": 0, "b": 0, "l": 0},
            bargroupgap=0.0,
            bargap=0.01,
            plot_bgcolor="white",
            paper_bgcolor="white",
        )

        # fig.show() ; break
        #
        fig.write_image(fig_name, scale=4)